# 1.2 Create Genie Space
Chat with your structured tables using natural language. Our [Genie Spaces](https://www.databricks.com/product/business-intelligence/ai-bi-genie) adds a LLM to your tables so it can reason from your chat request and generate SQL code to query the tables to return you an answer to your question

In [0]:
%pip install -U langgraph==0.3.2 langchain==0.3.19 langchain_experimental==0.3.4 databricks-sdk==0.44.1 mlflow>=2.20.3 databricks-langchain==0.3.0
dbutils.library.restartPython()

### [Set up](https://docs.databricks.com/aws/en/genie/set-up#-create-a-genie-space) a Genie Space via the UI
Then instantiate a `GenieAgent` to chat with your table [`retail_prod.agents.cust_service_data`](https://e2-demo-field-eng.cloud.databricks.com/explore/data/retail_prod/agents/cust_service_data?o=1444828305810485&activeTab=sample)

In [0]:
from databricks_langchain.genie import GenieAgent

# Get you Genie space ID from the URL 
# https://workspace_host/genie/rooms/<genie_id>/chats/...
genie_space_id =  "01f02f99c88f159c8828d2ee1043c198"
genie_agent = GenieAgent(genie_space_id, "Customer Service", description="Chat with your Customer Service table")

## Test your Genie Agent

In [0]:
query = "Which customer had the most interactions?"
response = genie_agent.invoke(
    {"messages": [{"role": "user", "content": query}]})
response['messages'][0].content

## [OPTIONAL] Register Genie as a UC Function
By registering Genie as a UC Function, you'll be able to access and re-use it from the Unity Catalog without having to re-instantiate `GenieAgent`.

In this workshop, we will use Genie as a `GenieAgent` in memory instead of a UC function although you can use either.

NB: the Genie space persists regardless

In [0]:
%sql
CREATE OR REPLACE FUNCTION yen_training.vs.genie_cust_data (
  question STRING COMMENT "Question to be answered using genie table",
  context STRING COMMENT "Provide relevant customer service request data to help answer the questionUse 'no relevant history' if there is nothing relevant to answer the question.")
RETURNS STRING
LANGUAGE SQL
COMMENT 'Chat with the table retail_prod.agents.cust_service_data which lists all the customers and their interactions with customer service.'
RETURN 
  SELECT _genie_query(
    "https://e2-demo-field-eng.cloud.databricks.com/",
    secret('hls-tokens', 'agents-demo'),
    "01f02f99c88f159c8828d2ee1043c198", -- genie space
    question, -- retrieved from function
    context -- retrieved from function
);
